# Hopfield network of pattern recognition

Hopfield networks are a kind of recurrent neural network that model auto-associative memory: the ability to recall a memory from just a partial piece of that memory.

In [ ]:
import os
import numpy as np
import imageio
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
# from skimage import img_as_ubyte

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)

np.random.seed(1)

Let's load in a meme. I'm partial to ['Deal with it'](https://a1.memecaptain.com/src_thumbs/22990.jpg).

In [ ]:
#deal = 2 * np.random.binomial(1,.5,size=(5,5)) - 1
#deal = imread('obama.png', mode="L")
deal = imread('small-deal-with-it-with-text.jpg', mode="L")
print(deal.shape)
deal = deal.astype(int)

In [ ]:
np.unique(deal)

To convert this to a 1 bit image, I convert everything darker than some threshold to black (1), and everything else to white (-1). Experimenting a bit with the particular image of the 'deal with it meme' that I have, a threshold of 80 seemed to work reasonably. The resulting image is still a bit rough around the edges, but it's recognizable.

In [ ]:
bvw_threshold = 80

deal[deal <= bvw_threshold] = -1
deal[deal >  bvw_threshold] = 1
deal = -deal
deal

In [ ]:
np.unique(deal)

In [ ]:
plt.imshow(deal, cmap='Greys', interpolation='nearest');

Now train the weights. **Whereas before we used Hebb's rule, now let's use the Storkey Learning Rule**. This rule has a few nice advantages over Hebb's rule: it allows the network to learn more patterns (the 'capacity is `n/sqrt(2*ln(n))` where `n` is the number of neurons in the network), its basins of attraction (to the stored patterns) are larger, the distribution of basin sizes is more even, and the shapes of the basins are more round. The weights at time `v` are:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e0b194a405a470e54aacef9e75ced89d02f60844)

where
![](https://wikimedia.org/api/rest_v1/media/math/render/svg/c0e9c85f5bbf569acdfc8dd7ab8cccb742a0f856)

and `n` is the number of neurons and $\epsilon$ is a bit (+1 or -1) of the pattern being trained at time `v`.

The second term of the rule is basically the Hebbian rule. The third and fourth terms basically account for the net input to neurons j and i using the current weights.

**To see the development/testing of the below implementation of the Storkey rule, see 'Hopfield Network of memes--Storkey Learning Rule development'.**

In [ ]:
def storkey_rule(pattern, old_weights=None):
    """
    pattern: 2-dimensional array
    old_weights: square array of length pattern.shape[0]*pattern.shape[1]
    """
    
    mem = pattern.flatten()    
    n = len(mem)
    
    if old_weights is None:
        old_weights = np.zeros(shape=(n,n))

    hebbian_term  = np.outer(mem,mem)
    
    net_inputs = old_weights.dot(mem)
    net_inputs = np.tile(net_inputs, (n, 1)) # repeat the net_input vector n times along the rows 
                                             # so we now have a matrix
    
    # h_i and h_j should exclude input from i and j from h_ij
    h_i = np.diagonal(old_weights) * mem # this obtains the input each neuron receives from itself
    h_i = h_i[:, np.newaxis]             # turn h_i into a column vector so we can subtract from hij appropriately
    
    h_j = old_weights * mem              # element-wise multiply each row of old-weights by mem    
    np.fill_diagonal(h_j,0)              # now replace the diagonal of h_j with 0's; the diagonal of h_j is the 
                                         # self-inputs, which are redundant with h_i; np.fill_diagonal modifies inplace
    
    hij = net_inputs - h_i - h_j
    
    post_synaptic  = hij * mem
    #pre_synaptic = post_synaptic.T
    pre_synaptic   = hij.T * mem[:, np.newaxis]
        
    new_weights = old_weights + (1./n)*(hebbian_term - pre_synaptic - post_synaptic)
    
    return new_weights

**This next cell can take a little while if the image is large. For an image of size 128x128, it takes a minute or two.**

In [ ]:
deal_weights = storkey_rule(deal, old_weights=None)
deal_weights

Now start with a noisy version of the image. We'll just flip a certain number of random pixels on each row of the image.

In [ ]:
def noisify(pattern, numb_flipped=30):

    noisy_pattern = pattern.copy()

    for idx, row in enumerate(noisy_pattern):
        choices = np.random.choice(range(len(row)), numb_flipped)
        noisy_pattern[idx,choices] = -noisy_pattern[idx,choices]
        
    return noisy_pattern

noisy_deal = noisify(pattern=deal)

In [ ]:
plt.imshow(noisy_deal, cmap='Greys', interpolation='nearest');

Now we can start with that, and use the weights to update it. We'll update the units asynchronously (one at a time), and keep track of the energy of the network every so often.

In [ ]:
def flow(pattern, weights, theta=0, steps = 50000):
    
    pattern_flat = pattern.flatten()

    if isinstance(theta, numbers.Number):
        thetas = np.zeros(len(pattern_flat)) + theta
    
    for step in range(steps):
        unit = np.random.randint(low=0, high=(len(pattern_flat)-1))
        unit_weights = weights[unit,:]
        net_input = np.dot(unit_weights,pattern_flat)
        pattern_flat[unit] = 1 if (net_input > thetas[unit]) else -1        
        #pattern_flat[unit] = np.sign(net_input)
        
        if (step % 10000) == 0:
            energy = -0.5*np.dot(np.dot(pattern_flat.T,weights),pattern_flat) + np.dot(thetas,pattern_flat)
            print("Energy at step {:05d} is now {}".format(step,energy))

    evolved_pattern = np.reshape(a=pattern_flat, newshape=(pattern.shape[0],pattern.shape[1]))
    
    return evolved_pattern

In [ ]:
steps = 50000
theta = 0

noisy_deal_evolved = flow(noisy_deal, deal_weights, theta = theta, steps = steps)

In [ ]:
plt.imshow(noisy_deal_evolved, cmap='Greys', interpolation='nearest');

Voila.

## Training the network on a second pattern

The cooler thing about the Hopfield networks is that they can encode multiple patterns (to a limit depending on the training regimen, and the number of units). So let's try another maymay.

I got the next meme from [here](https://68.media.tumblr.com/avatar_0f24a9a67d83_128.png), and then tweaked its levels in Mac's preview so that it'd translate nicely to a 1 bit (black or white) image.

In [ ]:
# woah = imread('woah.png', mode="L")
woah = imageio.imread('aang.jpg')[:,:,0]
woah = woah.astype(int)
woah[woah >= 1] = 1
woah[woah < 1] = -1
woah = -woah

In [ ]:
np.unique(woah)

In [ ]:
plt.imshow(woah, cmap='Greys', interpolation='nearest');

Cool. So now we make some weights for this image. The takes a little bit longer than the Hebbian learning rule when it is dealing with previous, nonzero weights.

In [ ]:
average_weights = storkey_rule(woah, old_weights=deal_weights)

In [ ]:
noisy_woah = noisify(pattern=woah, numb_flipped=15)
        
plt.imshow(noisy_woah, cmap='Greys', interpolation='nearest');

In [ ]:
recovered_woah = flow(noisy_woah, average_weights, theta = theta, steps = steps)

plt.imshow(recovered_woah, cmap='Greys', interpolation='nearest');

Now let's doublecheck that the average weights also still work for the 'deal with it' image.

In [ ]:
deal_recovered = flow(noisy_deal, average_weights, theta = theta, steps = steps)

plt.imshow(deal_recovered, cmap='Greys', interpolation='nearest');

Sweet. So *now* we can try something like feeding it a pattern that is halfway between the two patterns -- it should eventually settle into one of them! Who has greater meme strength!??!

In [ ]:
deal_with_neil = (woah + deal) / 2
print(np.unique(deal_with_neil))

I could force those 0 values to -1 or 1, but that biases the pattern towards deal and neil, respectively (at least, testing suggested this -- I think because Aang has more black pixels and Deal has more white pixels). So, I'll leave them in. I *could* probably solve this by randomly setting 0's to 1 or -1, but naw.

In [ ]:
#deal_with_neil[deal_with_neil == 0] = -1
#np.unique(deal_with_neil)

In [ ]:
plt.imshow(deal_with_neil, cmap='Greys', interpolation='nearest');

In [ ]:
recovered_deal_with_neil = flow(deal_with_neil, average_weights, theta = theta, steps = steps)

plt.imshow(recovered_deal_with_neil, cmap='Greys', interpolation='nearest');

*Assuming the cells/pixels of 0 were unaltered*, if you run that a few times, you'll notice that sometimes it settles on Neil, and sometimes it settles on Deal!!!

## Spurious patterns

Hopfield networks can also settle onto 'spurious patterns' (patterns that the network wasn't trained on). For each stored pattern `x`, `-x` is a spurious pattern. But also, any linear combination of the of the learned patterns can be a spurious pattern. So let's learn a third pattern, and then see the network stabilize on a simple combination of the three patterns.

In [ ]:
shrek = imageio.imread('shrek.jpg')
shrek = shrek.astype(int)

shrek_threshold = 200
shrek[shrek <  shrek_threshold] = -1
shrek[shrek >= shrek_threshold] = 1

shrek[120:,:] = 1

shrek = -shrek

plt.imshow(shrek, cmap='Greys', interpolation='nearest')
plt.show()

In [ ]:
flattened_shrek = shrek.flatten()

flatlen = len(flattened_shrek)

shrek_weights = np.outer(flattened_shrek,flattened_shrek) - np.identity(len(flattened_shrek))

In [ ]:
# average_weights = (woah_weights + deal_weights + shrek_weights) / 3

In [ ]:
noisy_shrek = noisify(pattern=shrek)
        
plt.imshow(noisy_shrek, cmap='Greys', interpolation='nearest')
plt.show()

In [ ]:
recovered_shrek = flow(noisy_shrek, average_weights, theta=theta, steps=steps)

plt.imshow(recovered_shrek, cmap='Greys', interpolation='nearest')
plt.show()

In [ ]:
recovered_woah = flow(noisy_woah, average_weights, theta=theta, steps=steps)

plt.imshow(recovered_woah, cmap='Greys', interpolation='nearest')
plt.show()

Okay, now let's make a spurious pattern. Any linear combination will do.

In [ ]:
spurious_meme = shrek + deal + woah
np.unique(spurious_meme)

In [ ]:
spurious_meme[spurious_meme > 0] = 1
spurious_meme[spurious_meme < 0] = -1

In [ ]:
plt.imshow(spurious_meme, cmap='Greys', interpolation='nearest')
plt.show()

Pretty noisy. Only Aang, and kiiiiinda the Deal with It, are visible. Now make a noisy version of that combination.

In [ ]:
noisy_spurious_meme = noisify(pattern=spurious_meme)
        
plt.imshow(noisy_spurious_meme, cmap='Greys', interpolation='nearest')
plt.show()

Beautifully noisy. Can barely see anything in it. But now if we start with that, and apply the weights, it should recover the spurious pattern!

In [ ]:
steps = 100000

recovered_spurious_meme = flow(noisy_spurious_meme, average_weights, theta=theta, steps=steps)

plt.imshow(recovered_spurious_meme, cmap='Greys', interpolation='nearest')
plt.show()

And it sure as heck did.

## Properties of the Storkey Learning Rule